## Custom Sentiment Analysis 
these first few blocks are me copy-pasting the https://realpython.com/python-nltk-sentiment-analysis/#using-nltks-pre-trained-sentiment-analyzer 
code and learning how it works

In [2]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("Wow, NLTK is really powerful!")

{'neg': 0.0, 'neu': 0.295, 'pos': 0.705, 'compound': 0.8012}

In [3]:
tweets = [t.replace("://", "//") for t in nltk.corpus.twitter_samples.strings()]

In [4]:
from random import shuffle

def is_positive(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia.polarity_scores(tweet)["compound"] > 0

shuffle(tweets)
for tweet in tweets[:10]:
    print(">", is_positive(tweet), tweet)

> True 30 minutes and counting just to pass through the EDSA AYALA tunnel... And am still not completely out. :(
> False RT @A_Liberty_Rebel: Hard-Left SWP/UAF/HnH violent thug &amp; BBC poppet Bunny La Roche just saw Miliband crushed on #BBCQT. She ain’t happy.. …
> True @onlytheshelley 6pm is when the doors open for everyone with  (VIP/general) tickets to get there seats early and watch support acts!!:)
> False As much as you staunch UKIP haters wouldn't like to admit, look at Farage right now. Talking sense, something most people don't understand 🙈
> False After watching election thing on BBC, Ed Miliband has simple trick of finding out name of person who asked question, instant rapport.
> True RT @sebeard98: younger Ed Milliband and Ed Balls look like a budget Ross and Chandler from F.R.I.E.N.Ds. http//t.co/GI09auE8Rb
> False @MrsCupcake79 @Redrho70 ??? ed miliband
> False @jeanlaydon would they let Nigel Farage in?
> False @McewenB despite Tory press Cameron will not win, again
> 

## my turn
Doing the same as above, but now with my corpus instead of tweets

In [5]:
#corpus as list of strings
import pandas as pd
df = pd.read_csv('C:\\Users\Connor\Desktop\\370\letterboxdCorpus\\all.csv')
reviews = df.reviews.tolist()
scoreless = []
for review in reviews:
    scoreless.append(review[5:])
print(scoreless[0:10])

['I highly recommend everyone get drunk with friends and watch this     ', "My introduction to John cena's absolutely delicious looking biceps     ", 'Not fast enough!     ', ' On march 24th 2023 Kanye West said he is no longer anti semitic because of this movie and Jonah Hill     ', "Why are old people having fun they're literally gonna die tomorrow 😭     ", 'Kids movie where dolls fight for their lives after the world ends in a terrifying way   somehow still less scary than happy feet     ', 'Yeah my ass was crying again my ass is always crying     ', 'Worst theater experience of my life fun movie tho     ', 'Wow wow wow wow wow WOW     ', ' Severe lack of solar systems but overall a good time     ']


In [6]:
shuffle(scoreless)
for review in scoreless[:10]:
    print(">", is_positive(review), review)

> True Tom Cruise and Ron Howard said freedom for Ireland! Kidman and Cruise follow up their Days of Thunder screen romance turned real life marriage with this nearly 2-and-a-half hour 19th-century Irish immigrant historical epic/class drama that is as dull as it is expensive. Kidman is actually quite good as the rebellious wealthy runaway and Cruise's star power has strong-armed roles that were maybe not conventionally suited to him into working before but an impoverished, accented bare-knuckle brawling farmer was maybe...     
> True In this one Godzilla beats the shit out of Ghidorah (again) and his new friend Gigan (who has blades for arms and a saw for abs) to stop alien cockroaches from building an amusement park or something. Then he swims away into the sunrise with his good pal Anguirus to a spaghetti western theme song.     
> True This movie has the most weirdly inconsistent animation I've ever seen, there are some scenes that are straight-up some of the most gorgeous things 

I honestly don't mind the supposed ~60% accuracy. I do like the results.

## generating top 100s

In [7]:
unwanted = nltk.corpus.stopwords.words("english")
unwanted.extend([w.lower() for w in nltk.corpus.names.words()])
#
#here's my own injection, which makes a list of words in positive reviews and a list
#of negative reviews, so that I don't need to use the nltk.corpus movie reviews
pos_category = []
neg_category = []
for review in scoreless:
    words = review.split()
    if is_positive(review): 
        for word in words:
            pos_category.append(word)
    else:
        for word in words:
            neg_category.append(word)


def skip_unwanted(pos_tuple):
    word, tag = pos_tuple
    if not word.isalpha() or word in unwanted:
        return False
    if tag.startswith("NN"):
        return False
    return True

#here's where it trains off my stuff rather than nltk movie reviews
positive_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(pos_category)
)]
negative_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(neg_category)
)]

In [8]:
positive_fd = nltk.FreqDist(positive_words)
negative_fd = nltk.FreqDist(negative_words)

common_set = set(positive_fd).intersection(negative_fd)

for word in common_set:
    del positive_fd[word]
    del negative_fd[word]

top_100_positive = {word for word, count in positive_fd.most_common(100)}
top_100_negative = {word for word, count in negative_fd.most_common(100)}
print("POSTIVE:")
print(top_100_positive)
print("NEGATIVE:")
print(top_100_negative)

POSTIVE:
{'cringe', 'Preparing', 'WANT', 'intelligent', 'Easily', 'curly', 'richly', 'undervalued', 'euphoric', 'obscure', 'Best', 'emerges', 'thrilled', 'subverts', 'uncharted', 'revered', 'kissing', 'subscribing', 'breezy', 'owed', 'thankful', 'transcends', 'Bible', 'sporadically', 'supportive', 'hopeful', 'impoverished', 'Subscribing', 'hong', 'sugary', 'perceive', 'insistently', 'bat', 'enthusiastic', 'grateful', 'disappoint', 'celebrate', 'exploded', 'eagerly', 'pleasantly', 'maniacal', 'seamless', 'mandatory', 'gain', 'cleanest', 'earth', 'kidding', 'extend', 'automatic', 'riotous', 'ecstatically', 'rebellious', 'improvisational', 'ranked', 'microcosmic', 'vibe', 'blessed', 'hagiographic', 'entered', 'eps', 'mined', 'smooth', 'panoramic', 'joyful', 'united', 'remained', 'sweetie', 'smile', 'tweeted', 'minding', 'credits', 'satisfied', 'pleasing', 'ground', 'purple', 'subconsciously', 'table', 'captivated', 'pls', 'intricately', 'inclusive', 'inspirational', 'breathlessly', 'treas

So, there are the rough top 100 positive and negative words in my corpus. Apart from a few questionable results here and there, I'd consider it mostly good.

Here's an idea. I'm going to redefine my pos/neg_category variables using the user scores attached to reviews. I'm hoping it gives me more accurate evaluations than the polarity scores

In [31]:
import nltk
from nltk.tokenize import word_tokenize

#8s, 9s, and 10/10s
good_movies = []
#3s, 2s, and 1/10s
bad_movies = []
#var for later
sentences = []
for review in reviews:
    if review.startswith("10") or review.startswith("9") or review.startswith("8"):
        lower = review[5:].lower()
        tokens = word_tokenize(lower)
        for token in tokens:
            good_movies.append(token)
    if review.startswith("1/") or review.startswith("2") or review.startswith("3"):
        lower = review[5:].lower()
        tokens = word_tokenize(lower)
        for token in tokens:
            bad_movies.append(token)
    lower2 = review[5:].lower()
    tokens2 = word_tokenize(lower2)
    sentences.append(tokens2)
print(bad_movies[0:50])

['i', 'highly', 'recommend', 'everyone', 'get', 'drunk', 'with', 'friends', 'and', 'watch', 'this', 'my', 'introduction', 'to', 'john', 'cena', "'s", 'absolutely', 'delicious', 'looking', 'biceps', 'i', 'like', 'that', 'one', 'robert', 'de', 'niro', 'comedy', 'where', 'he', 'shows', 'his', 'cock', 'and', 'fucks', 'like', 'crazy', 'this', 'is', 'not', 'that', 'one', 'tho', 'i', 'know', 'a', 'guy', 'named', 'aksel']


I'm too lazy to go up and make a function

In [28]:
#now processing new stuff see line 4 and 8
positive_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(good_movies)
)]
negative_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(bad_movies)
)]
positive_fd = nltk.FreqDist(positive_words)
negative_fd = nltk.FreqDist(negative_words)

common_set = set(positive_fd).intersection(negative_fd)

for word in common_set:
    del positive_fd[word]
    del negative_fd[word]

top_100_positive = {word for word, count in positive_fd.most_common(100)}
top_100_negative = {word for word, count in negative_fd.most_common(100)}
print("POSTIVE:")
print(top_100_positive)
print("NEGATIVE:")
print(top_100_negative)

POSTIVE:
{'sensual', 'finds', 'layered', 'warm', 'fractured', 'experimental', 'allows', 'gorgeously', 'thrilling', 'melancholic', 'experience', 'apocalyptic', 'delicate', 'moves', 'vividly', 'atmosphere', 'operating', 'adds', 'subscribing', 'sees', 'captures', 'marvelous', 'expertly', 'beautifully', 'tiny', 'stronger', 'terrified', 'vicious', 'phenomenal', 'indelible', 'twin', 'existential', 'shaking', 'elemental', 'breaking', 'builds', 'adding', 'detailed', 'shown', 'scariest', 'grateful', 'underrated', 'trapped', 'ep', 'inherent', 'loving', 'claustrophobic', 'flawed', 'flows', 'subtle', 'noir', 'uses', 'pulls', 'allow', 'gentle', 'abstract', 'essential', 'detective', 'harrowing', 'harsh', 'invented', 'ghostly', 'neon', 'vivid', 'exquisite', 'fitting', 'revolutionary', 'sudden', 'vital', 'tight', 'nightmarish', 'gothic', 'touching', 'devastating', 'gleefully', 'grew', 'tense', 'empathetic', 'hearing', 'rewatch', 'surreal', 'familial', 'hysterical', 'wrapped', 'magnificent', 'masterful

Even cooler results. Although maybe affected by a smaller sample size, since I'm seeing the infrequently used "fuckahhhhhhhhh" in the negative.

So my best idea to find more stuff with word embeddings is to process my positive and negative words and see what pops up frequently in .most_similar calls

In [34]:
from gensim.models import Word2Vec
print(len(sentences))
model = Word2Vec(sentences = sentences,
                 window = 2, 
                 min_count =1,
                 vector_size = 5)

25344


In [54]:
#similar_words = model.wv.most_similar('sensual', topn= 10)
# ^ list of tuples. first word = [0][0]
#making a dictionary in the (word, count) format
near_positive = {}
for word in top_100_positive:
    sim = model.wv.most_similar(word, topn= 500)
    for i in range(500):
        synonym = sim[i][0]
        #print(word)
        if synonym in near_positive:
            near_positive[synonym] += 1
        else:
            near_positive[synonym] = 1
sorted_pos = {k: v for k, v in sorted(near_positive.items(), key=lambda item: item[1], reverse=True)}
print("WORDS \"SIMILAR TO\" POSITIVE WORDS:")
print(sorted_pos)

WORDS "SIMILAR TO" POSITIVE WORDS:
{'edits': 30, 'department': 29, 'abercrombie': 28, 'mediocre': 28, 'ambitions': 27, 'showcase': 27, 'natural': 27, 'devito': 27, 'unforgiven': 27, 'puppet': 27, 'claustrophobic': 27, 'zooms': 27, 'manic': 26, 'philadelphia': 26, 'bold': 26, 'opens': 26, 'dangerous': 26, 'coon': 26, 'tool': 26, 'soft': 26, 'kaiju': 26, 'apartment': 26, 'expressive': 26, 'grand': 26, 'messy': 25, 'depicting': 25, 'gabriel': 25, 'surface': 25, 'convention': 25, 'ex-soviet': 25, 'blunt': 25, 'marketing': 25, 'interviews': 25, 'shitty': 25, 'loop': 25, 'aspects': 25, 'lo-fi': 25, 'oppressive': 25, 'blurring': 25, 'superficiality': 24, 'comically': 24, 'criticism': 24, 'melancholy': 24, 'reshoots': 24, 'whodunnit': 24, 'fits': 24, 'random': 24, 'price': 24, 'storytelling': 24, 'recorded': 24, 'incident': 24, 'gentle': 24, 'famous': 24, 'craft': 24, 'shock': 24, 'fallacies': 24, 'truthsayer': 24, 'abilities': 24, 'sustained': 24, 'neo-noir': 24, '90s': 24, 'industry': 23, 'i

In [53]:
near_negative = {}
for word in top_100_negative:
    sim = model.wv.most_similar(word, topn= 500)
    for i in range(500):
        synonym = sim[i][0]
        #print(word)
        if synonym in near_negative:
            near_negative[synonym] += 1
        else:
            near_negative[synonym] = 1
sorted_neg = {k: v for k, v in sorted(near_negative.items(), key=lambda item: item[1], reverse=True)}
print("WORDS \"SIMILAR TO\" NEGATIVE WORDS:")
print(sorted_neg)

WORDS "SIMILAR TO" NEGATIVE WORDS:
{'dry': 12, 'reaches': 12, 'stain': 11, 'annoying': 11, 'introducing': 11, 'hyper-accomplished': 11, 'provocative': 11, 'self-consciously': 11, 'admirable': 11, 'heist': 11, 'slow-motion': 10, 'ghost': 10, 'kirby': 10, 'earnest': 10, 'half-quizzical': 10, 'odd': 10, 'remarkably': 10, 'issue': 10, 'boxing': 10, 'utterly': 10, 'date': 10, 'absurd': 10, 'standout': 10, 'absorbing': 10, 'undeniably': 10, 'creepy': 10, 'tacked-on': 9, 'vehicle': 9, 'demonstrably': 9, 'corny': 9, 'bud': 9, 'crackles': 9, 'stupendous': 9, 'minor': 9, 'spectacular': 9, 'dated': 9, 'narratively': 9, 'heartbreaking': 9, 'actor': 9, 'lifeless': 9, 'zo': 9, 'remorse': 9, 'spectacle': 9, 'adaptation': 9, 'uneven': 9, 'lead': 9, 'contributing': 9, 'derivative': 9, 'frustrating': 9, 'territory': 9, 'shocking': 9, 'overlong': 9, 'astonishing': 9, 'airless': 9, 'sleepy': 9, 'quiet': 9, 'superhero': 9, 'stands': 9, 'immersive': 9, 'scumbag': 9, 'thin': 9, 'reveal': 9, 'colours': 9, 'ex

These results kinda suck, but improve some as i turn up the topn parameter. It's likely best to ignore most of the keys with value 1.